In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os

In [ ]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

letter_list = list(alphabet)
letter_list

In [10]:
url_template = 'https://www.pro-football-reference.com/players/{}/'

def get_WR(letter):
    url = url_template.format(letter) # This formats the url by letter

    response = requests.get(url)
    time.sleep(8)

    soup = BeautifulSoup(response.text, 'html.parser')

    players = []
    div_players = soup.find('div', class_="section_content", id="div_players")
    
    # Loop through p tags
    for p in div_players.find_all('p'):
        meta = p.get_text()
        position = str(meta.split('(')[1].split(')')[0])
        is_wr = 1 if 'WR' in position else 0
        year_span = meta.split(')')[1].strip()
        start_year = int(year_span.split('-')[0])
        hyper_link = p.find('a')['href']
        player_link = f'https://www.pro-football-reference.com{hyper_link}'
        if is_wr == 1 and start_year >= 2018:
            players.append([meta, position, year_span, player_link])
    df = pd.DataFrame(players, columns = ['meta', 'position', 'year_span', 'player_link'])
    return df 

all_players = pd.concat([get_WR(letter) for letter in letter_list], ignore_index=True)
all_players.to_csv("wide_receiver_links.csv", index = False)

In [9]:
# Define the output folder
output_folder = 'Wr_Players'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the CSV with player links
wr = pd.read_csv('wide_receiver_links.csv')

# Scrape the tables off the player links
for index, player_link in enumerate(wr['player_link']):
    try:
        response = requests.get(player_link)
        time.sleep(8)

        soup = BeautifulSoup(response.text, 'html.parser')

        table = soup.find_all('table')[0]

        reg_df = []

        for i, row in enumerate(table.find_all('tr')[2:]):
            try:
                player_info = row.find('td', attrs={'data-stat': 'player'})
                name = player_info.a.get_text()
                position = 'WR'

                # Grab the player stats
                tdf = pd.read_html(player_link)[0]

                # Get rid of MultiIndex, keep last row
                tdf.columns = tdf.columns.get_level_values(-1)

                # Drop unnecessary columns
                tdf = tdf.iloc[:, [1, 3, 4] + list(range(19, tdf.shape[1]))]

                # Add other info
                tdf['Name'] = name
                tdf['Position'] = position

                # Append the DataFrame to reg_df
                reg_df.append(tdf)

                # Create and save an individual CSV file for the player
                file_path = os.path.join(output_folder, f'{name}.csv')
                tdf.to_csv(file_path, index=False)

            except Exception as e:
                print(f"Error for player at index {index}: {str(e)}")

    except Exception as e:
        print(f"Error for player at index {index}: {str(e)}")

# Concatenate all player DataFrames into one DataFrame if needed
if reg_df:
    reg_df = pd.concat(reg_df, ignore_index=True)
    reg_df.head()

print("Scraping and CSV file creation completed.")

Error for player at index 0: 'NoneType' object has no attribute 'a'
Error for player at index 1: 'NoneType' object has no attribute 'a'
Error for player at index 1: 'NoneType' object has no attribute 'a'
Error for player at index 1: 'NoneType' object has no attribute 'a'
Error for player at index 1: 'NoneType' object has no attribute 'a'


KeyboardInterrupt: 